In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [2]:
#Download training data from open datasets
training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download=True,
    transform=ToTensor(),
)
test_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download=True,
    transform=ToTensor(),
)

In [3]:
#datasets to dataloader
batch_size = 64
train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)

for X,y in test_dataloader:
    print("shape of X [N,C,H,W] ", X.shape)
    print("shape of y: ", y.shape, y.dtype)
    break

shape of X [N,C,H,W]  torch.Size([64, 1, 28, 28])
shape of y:  torch.Size([64]) torch.int64


In [5]:
#Get cpu or gpu device for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

#Define model by creating a class that inherits from nn.modules
#__init__: define layers of network
#forward: specify how data will pass through the network
#The network cost 1005MiB GPU Memory
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
#We can move our model to GPU
model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
#training
#set loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)
#train process
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred,y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10000 ==0:
            loss,current = loss.item(),batch*len(X)
            print(f"loss:{loss:>7f} [{current:>5d}/{size:>5d}]")
            

In [15]:
# testing 
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    print(device)
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Acurracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")  
        

In [8]:
# Training Process
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-----------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-----------
loss:2.298782 [    0/60000]
Test Error: 
 Acurracy: 49.8%, Avg loss: 2.150233 

Epoch 2
-----------
loss:2.152166 [    0/60000]
Test Error: 
 Acurracy: 61.5%, Avg loss: 1.871408 

Epoch 3
-----------
loss:1.899614 [    0/60000]
Test Error: 
 Acurracy: 63.1%, Avg loss: 1.491531 

Epoch 4
-----------
loss:1.561526 [    0/60000]
Test Error: 
 Acurracy: 64.4%, Avg loss: 1.224332 

Epoch 5
-----------
loss:1.308788 [    0/60000]
Test Error: 
 Acurracy: 65.8%, Avg loss: 1.063105 

Done!


In [9]:
# model saving
torch.save(model.state_dict(),"model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [6]:
# model loading
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [18]:
#Get cpu or gpu device for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
model.to(device)
test(test_dataloader, model, loss_fn)

Using cuda device
cuda
Test Error: 
 Acurracy: 65.8%, Avg loss: 1.063105 



In [ ]:
# prediction
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

In [19]:
torch.cuda.empty_cache()